## Setup up CUDA for PyTorch
#### *Must download CUDA drive from https://developer.nvidia.com/cuda-downloads and make a venv after in anaconda, not before. Then run the kernel below. 

In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [4]:
!nvidia-smi

Wed Apr  3 11:27:38 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.23                 Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   62C    P8             18W /   81W |     786MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Setup RoboFlow

In [1]:
!pip install roboflow --quiet

## Setup SAM

In [3]:
!pip install git+https://github.com/facebookresearch/segment-anything.git

  Cloning https://github.com/facebookresearch/segment-anything.git to c:\users\richa_0\appdata\local\temp\pip-req-build-q5ostivs
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36830 sha256=6bd36929ca9babdebffb454221eb495d81a78e706933e227f5d90e015010e448
  Stored in directory: C:\Users\richa_0\AppData\Local\Temp\pip-ephem-wheel-cache-c_kn2u_r\wheels\10\cf\59\9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git 'C:\Users\richa_0\AppData\Local\Temp\pip-req-build-q5ostivs'


In [4]:
!pip install -q  supervision jupyter_bbox_widget roboflow dataclasses-json

#### Download SAM weights

In [ ]:
!curl -O https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

## Setup Computer and Dependencies

In [1]:
import torch
print(torch.__version__)
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

2.2.0+cu118
cuda:0


In [2]:
MODEL_TYPE = "vit_h"
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
sam = sam_model_registry[MODEL_TYPE](checkpoint=r"C:\Users\richa_0\Documents\Coral Research\Machine Learning Models\sam_vit_h_4b8939.pth").to(device=DEVICE)

In [5]:
rf = Roboflow(api_key="SsfDz85MaMV4PPpSPJLO")
project = rf.workspace().project("coral-finder")
model = project.version(3).model

loading Roboflow workspace...
loading Roboflow project...


#### Add more dependencies

In [51]:
import numpy as np
from PIL import Image
from PIL.ExifTags import TAGS
import matplotlib.pyplot as plt
import matplotlib.patches as patches 
import torchvision
import os
import ultralytics
from ultralytics import YOLO
from matplotlib import pyplot as plt
import pandas as pd
import torch
import supervision as sv
from roboflow import Roboflow
import imageio.v3 as iio

## Train Custom YOLO V8

In [4]:
import yaml
model = YOLO('yolov8n.pt')
name = "50_epochs-"
project = "C:/Users/richa_0/Documents/Coral Research/Machine Learning Models/coral finder.v4i.yolov8/results"

In [5]:
results = model.train(data=r"C:\Users\richa_0\Documents\Coral Research\Machine Learning Models\coral finder.v4i.yolov8\data.yaml",
                      project=project,
                      name=name,
                      epochs=50,
                      patience=0, #I am setting patience=0 to disable early stopping.
                      batch=4,
                      imgsz=640)

New https://pypi.org/project/ultralytics/8.2.11 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.11 🚀 Python-3.10.13 torch-2.2.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\richa_0\Documents\Coral Research\Machine Learning Models\coral finder.v4i.yolov8\data.yaml, epochs=50, time=None, patience=0, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=C:/Users/richa_0/Documents/Coral Research/Machine Learning Models/coral finder.v4i.yolov8/results, name=50_epochs-, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half

train: Scanning C:\Users\richa_0\Documents\Coral Research\Machine Learning Models\coral finder.v4i.yolov8\train\labels... 609 images, 0 backgrounds, 0 corrupt: 100%|██████████| 609/609 [00:00<00:00, 1521.02it/s]


train: New cache created: C:\Users\richa_0\Documents\Coral Research\Machine Learning Models\coral finder.v4i.yolov8\train\labels.cache


val: Scanning C:\Users\richa_0\Documents\Coral Research\Machine Learning Models\coral finder.v4i.yolov8\valid\labels... 174 images, 0 backgrounds, 0 corrupt: 100%|██████████| 174/174 [00:00<00:00, 859.91it/s]

val: New cache created: C:\Users\richa_0\Documents\Coral Research\Machine Learning Models\coral finder.v4i.yolov8\valid\labels.cache


...

      50/50       3.5G     0.5689     0.2896      0.975          1        640: 100%|██████████| 153/153 [00:10<00:00, 15.27it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:01<00:00, 20.07it/s]

                   all        174        174      0.987      0.977      0.991      0.833



50 epochs completed in 0.176 hours.
Optimizer stripped from C:\Users\richa_0\Documents\Coral Research\Machine Learning Models\coral finder.v4i.yolov8\results\50_epochs-\weights\last.pt, 6.3MB
Optimizer stripped from C:\Users\richa_0\Documents\Coral Research\Machine Learning Models\coral finder.v4i.yolov8\results\50_epochs-\weights\best.pt, 6.3MB

Validating C:\Users\richa_0\Documents\Coral Research\Machine Learning Models\coral finder.v4i.yolov8\results\50_epochs-\weights\best.pt...
Ultralytics YOLOv8.1.11 🚀 Python-3.10.13 torch-2.2.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:01<00:00, 17.76it/s]


                   all        174        174      0.987      0.977      0.991      0.833
Speed: 0.3ms preprocess, 1.7ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to C:\Users\richa_0\Documents\Coral Research\Machine Learning Models\coral finder.v4i.yolov8\results\50_epochs-


## Define Methods

In [53]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.2]) #0.6
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375): #Extra method for the SAM inputs that we don't use
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_box(box, ax): #For debugging purposes to check if YOLO V8 is working
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    rect = plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2)
    ax.add_patch(rect)

def get_area(mask): #Prints the mask onto the image in a blue color
    true_count = np.count_nonzero(mask)
    return true_count
"""
def get_physical(image):
    physical_size_x = 1
    physical_size_y = 1
    exifdata = image.getexif()
    for tagid in exifdata:
        tagname = TAGS.get(tagid, tagid)
        if tagname == "ImageDescription":
            try:
                value = exifdata.get(tagid)
                start_index_x = value.find('PhysicalSizeX="') + len('PhysicalSizeX="')
                end_index_x = value.find('"', start_index_x)
                physical_size_x = value[start_index_x:end_index_x]
                start_index_x_unit = value.find('PhysicalSizeXUnit="') + len('PhysicalSizeXUnit="')
                end_index_x_unit = value.find('"', start_index_x_unit)
                physical_size_x_unit = value[start_index_x_unit:end_index_x_unit]
                start_index_y = value.find('PhysicalSizeY="') + len('PhysicalSizeY="')
                end_index_y = value.find('"', start_index_y)
                physical_size_y = value[start_index_y:end_index_y]
                start_index_y_unit = value.find('PhysicalSizeYUnit="') + len('PhysicalSizeYUnit="')
                end_index_y_unit = value.find('"', start_index_y_unit)
                physical_size_y_unit = value[start_index_y_unit:end_index_y_unit]
            except Exception as e:
                print(f'Failed to process {image}: {e}')
                continue
    return float(physical_size_x), float(physical_size_y)
"""

def get_physical(image_path):
    physical_size_x = 1
    physical_size_y = 1
    try:
        metadata = iio.immeta(image_path)
        physical_size_x = metadata.get('pixelsizex', 'N/A')
        physical_size_y = metadata.get('pixelsizey', 'N/A')
        physical_size_x = float(("%.17f" % physical_size_x).rstrip('0').rstrip('.')) * 10 ** 6
        physical_size_y = float(("%.17f" % physical_size_y).rstrip('0').rstrip('.')) * 10 ** 6
    except Exception as e:
        print(f'Failed to retrieve metadata from {image_path}: {e}')
    return float(physical_size_x), float(physical_size_y)

In [57]:
def execute_block1(image_path):
    image = Image.open(image_path)
    image_np = np.array(image)
    height, width, channels = image_np.shape
    model = YOLO("C:/Users/richa_0/Documents/Coral Research/Machine Learning Models/coral finder.v4i.yolov8/results/50_epochs-/weights/best.pt") #Coral_Area_Generator_V1.yolov8
    results = model.predict(image, verbose=False)
    for result in results:
        boxes = result.boxes
    if boxes.xyxy.tolist():
        bbox = boxes.xyxy.tolist()[0]
    else:
        bbox = [0, width, 0, height]
    return image, bbox

def execute_block2(image, bbox):
    predictor = SamPredictor(sam)
    predictor.set_image(np.array(image))
    input_box = np.array(bbox)
    masks, _, _ = predictor.predict(
        point_coords=None,
        point_labels=None,
        box=input_box[None, :],
    multimask_output=False,
    )
    return masks

def execute_block3(image, bbox, masks, annotated_path, index):
    input_box = np.array(bbox)
    image_np = np.array(image)
    height, width, channels = image_np.shape
    scale_factor = 100
    fig, ax = plt.subplots(figsize=(width / scale_factor, height / scale_factor))
    ax.set_position([0, 0, 1, 1])
    ax.set_axis_off()
    ax.imshow(image, aspect='auto')
    show_mask(masks[0], ax)
    show_box(input_box, ax)
    fig.savefig(annotated_path, dpi=100)
    plt.close(fig)
    pixels = get_area(masks[0])
    return pixels

## Define Important Variables

In [27]:
folder_read_paths = [
    "C:/Users/richa_0/Documents/Coral Research/Images/Original_Raw_95_TIF",
    #"/content/gdrive/MyDrive/Coral Area Generator V1 Folder/1-100/",
    #"/content/gdrive/MyDrive/Coral Area Generator V1 Folder/101-200/",
    #"/content/gdrive/MyDrive/Coral Area Generator V1 Folder/201-300/",
    #"/content/gdrive/MyDrive/Coral Area Generator V1 Folder/301-400/",
    #"/content/gdrive/MyDrive/Coral Area Generator V1 Folder/401-500/",
    #"/content/gdrive/MyDrive/Coral Area Generator V1 Folder/501-600/",
    #"/content/gdrive/MyDrive/Coral Area Generator V1 Folder/601-700/",
    #"/content/gdrive/MyDrive/Coral Area Generator V1 Folder/701-783/"
]
folder_write_path = "C:/Users/richa_0/Documents/Coral Research/Code Output/"
csv_path = folder_write_path + "coral_areas_output.csv"
columns = ["Folder", "Image Name", "Pixel Area", "Pixel Size X", "Pixel Size Y", "µm^2"]

## Run Area Generator

In [58]:
for index, folder in enumerate(folder_read_paths):
    folder_name = os.path.basename(folder.rstrip('/'))
    for image_file in os.listdir(folder):
        if image_file.endswith('.tif'):
            image_path = os.path.join(folder, image_file)
            annotated_path = os.path.join(folder_write_path, image_file)
            image, bbox = execute_block1(image_path)
            masks = execute_block2(image, bbox)
            pixels = execute_block3(image, bbox, masks, annotated_path, index)
            pixel_size_x, pixel_size_y = get_physical(image_path)
            area = pixel_size_x * pixel_size_y * pixels
            new_row = {"Folder": folder_name, "Image Name": image_file, "Pixel Area": pixels, "Pixel Size X": pixel_size_x, "Pixel Size Y": pixel_size_y, "µm^2": area}
            new_row_df = pd.DataFrame([new_row], columns=columns)
            new_row_df.to_csv(csv_path, mode='a', header=not os.path.exists(csv_path), index=False)
            print("Folder: " + folder_name + "\n" + "Image Name: " + image_file + "\n" + "Bounding Box: " + str(bbox) + "\n" + "Pixels: " + str(pixels) + "\n" + "Pixel Size X: " + str(pixel_size_x) + "\n" + "Pixel Size Y: " + str(pixel_size_y) + "\n" + "Area: " + str(area) + " µm^2" + "\n")

Folder: Original_Raw_95_TIF
Image Name: 101_1_20230925.tif
Bounding Box: [936.5484619140625, 655.83935546875, 1296.239501953125, 989.2706298828125]
Pixels: 86995
Pixel Size X: 2.15891138174
Pixel Size Y: 2.15891138174
Area: 405474.8523241958 µm^2

Folder: Original_Raw_95_TIF
Image Name: 101_2_20230925.tif
Bounding Box: [932.1730346679688, 631.3567504882812, 1256.83203125, 992.5339965820312]
Pixels: 90084
Pixel Size X: 2.15891138174
Pixel Size Y: 2.15891138174
Area: 419872.3673403397 µm^2

Folder: Original_Raw_95_TIF
Image Name: 101_3_20230925.tif
Bounding Box: [956.2066650390625, 625.7059936523438, 1257.623046875, 979.8914184570312]
Pixels: 83890
Pixel Size X: 2.15891138174
Pixel Size Y: 2.15891138174
Area: 391002.7629343846 µm^2

Folder: Original_Raw_95_TIF
Image Name: 101_4_20230925.tif
Bounding Box: [803.6498413085938, 568.1477661132812, 1143.5340576171875, 935.6941528320312]
Pixels: 97438
Pixel Size X: 2.15891138174
Pixel Size Y: 2.15891138174
Area: 454148.61383717443 µm^2

Folder: